In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


scrape the debate

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Specify the URL of the webpage
url = 'https://www.presidency.ucsb.edu/documents/presidential-debate-atlanta-georgia'  # Replace with the URL you want to scrape

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize lists to store speakers and their corresponding text
    speakers = []
    texts = []

    # Find all paragraphs and process them
    current_speaker = None
    current_text = []

    for p in soup.find_all('p'):
        # Check if the paragraph has a <b> tag (indicating a speaker)
        b_tag = p.find('b')
        if b_tag:
            # If there's already a speaker and text, save the previous data
            if current_speaker and current_text:
                speakers.append(current_speaker)
                texts.append(' '.join(current_text))
                current_text = []

            # Update the current speaker and clean up the text
            current_speaker = b_tag.get_text(strip=True).replace(':', '')
            # Add the text of the paragraph, excluding the speaker's name
            p_text = p.get_text(strip=True)
            speaker_text = p_text.replace(b_tag.get_text(strip=True), '').strip()
            current_text.append(speaker_text)
        else:
            # If there's no <b> tag, just add the text to the current speaker's text
            if current_speaker:
                current_text.append(p.get_text(strip=True))

    # After the loop, add the last speaker's text
    if current_speaker and current_text:
        speakers.append(current_speaker)
        texts.append(' '.join(current_text))

    # Create a DataFrame and save it as CSV
    df = pd.DataFrame({
        'Speaker': speakers,
        'Text': texts
    })

    # Save the DataFrame to a CSV file
    df.to_csv('debate_transcript.csv', index=False)

    print("CSV file created successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

CSV file created successfully.


# scrape https://www.presidency.ucsb.edu

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL for the main page with all the links
base_url = 'https://www.presidency.ucsb.edu/documents/app-categories/spoken-addresses-and-remarks/presidential/state-the-union-addresses?items_per_page=3&page=1'

# Send a GET request to fetch the content of the base page
response = requests.get(base_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the main page content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize lists to store the titles and content
    titles = []
    contents = []

    # Find all the links to the individual documents
    for link in soup.find_all('a', href=True, text=True):
        if '/documents/the' in link['href']:  # Filter to make sure we get the correct links
            doc_url = f"https://www.presidency.ucsb.edu{link['href']}"

            # Request the individual document page
            doc_response = requests.get(doc_url)
            if doc_response.status_code == 200:
                doc_soup = BeautifulSoup(doc_response.content, 'html.parser')

                # Extract the title from the h1 tag
                title = doc_soup.find('h1').get_text(strip=True)
                titles.append(title)

                # Extract and combine all paragraph content
                content_paragraphs = doc_soup.find('div', class_='field-docs-content').find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in content_paragraphs])
                contents.append(content)

    # Create a DataFrame with the extracted data
    df = pd.DataFrame({
        'Title': titles,
        'Content': contents
    })

    # Save the DataFrame to a CSV file
    df.to_csv('/content/drive/My Drive/RAG Debate Project Lab semester 8 /trump_state_of_the_union.csv', index=False)

    print("CSV file created successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


CSV file created successfully.


<ipython-input-15-78e3bb9da553>:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for link in soup.find_all('a', href=True, text=True):


In [ ]:
# prompt: open trump_news_conferences.json andsave  to /content/drive/My Drive/RAG Debate Project Lab semester 8 as a json

import json

# Open the JSON file
with open('trump_news_conferences.json', 'r') as f:
  data = json.load(f)

# Save the JSON data to a new file
with open('/content/drive/My Drive/RAG Debate Project Lab semester 8/trump_news_conferences.json', 'w') as f:
  json.dump(data, f, indent=4)


In [ ]:
import requests
from bs4 import BeautifulSoup
import json

# Base URL for the main page with all the links
base_url = 'https://www.presidency.ucsb.edu/documents/app-categories/presidential/news-conferences?items_per_page=60&page=1'

# Send a GET request to fetch the content of the base page
response = requests.get(base_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the main page content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize a list to store the data
    data = []

    # Find all the links to the individual documents
    for link in soup.find_all('a', href=True, text=True):
        if '/documents/the' in link['href']:  # Filter to make sure we get the correct links
            doc_url = f"https://www.presidency.ucsb.edu{link['href']}"

            # Request the individual document page
            doc_response = requests.get(doc_url)
            if doc_response.status_code == 200:
                doc_soup = BeautifulSoup(doc_response.content, 'html.parser')

                # Extract the title from the h1 tag
                title = doc_soup.find('h1').get_text(strip=True)

                # Extract and combine all paragraph content
                content_paragraphs = doc_soup.find('div', class_='field-docs-content').find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in content_paragraphs])

                # Append to the data list
                data.append({
                    'Title': title,
                    'Content': content
                })

    # Save the data to a JSON file
    with open('/content/drive/My Drive/RAG Debate Project Lab semester 8/Trump/trump_news_conferences.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

    print("JSON file created successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


<ipython-input-50-e643069a53b6>:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for link in soup.find_all('a', href=True, text=True):


JSON file created successfully.


SOTU

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os

# Base URL for constructing full links
base_url = 'https://www.presidency.ucsb.edu'

# URL of the main page with the list of speeches
main_url = 'https://www.presidency.ucsb.edu/documents/app-categories/spoken-addresses-and-remarks/presidential/state-the-union-addresses?items_per_page=3&page=1'

# Directory to save the JSON file
save_dir = '/content/drive/My Drive/RAG Debate Project Lab semester 8/'
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Step 1: Extract links from the main page
response = requests.get(main_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all speech links
links = soup.select('a[href^="/documents/address-before-joint-session-the-congress-the-state-the-union-"]')

# Step 2: Visit each link and extract data
data = []

for link in links:
    href = link['href']
    full_url = base_url + href
    page_response = requests.get(full_url)
    page_soup = BeautifulSoup(page_response.text, 'html.parser')

    # Extract the title (h1)
    title = page_soup.find('h1').get_text(strip=True)

    # Extract all paragraphs and concatenate them
    paragraphs = page_soup.find_all('p')
    content = ' '.join([para.get_text(strip=True) for para in paragraphs])

    # Print all content
    print(content)

    # Store the result
    data.append({'title': title, 'content': content})

# Step 3: Save the data to a JSON file
json_file_path = os.path.join(save_dir, 'trump_state_of_the_union_speeches.json')
with open(json_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"Data extraction completed and saved to '{json_file_path}'")


AboutSearch  The President.Thank you very much. Thank you. Thank you very much. Madam Speaker, Mr. Vice President, Members of Congress, the First Lady of the United States, and my fellow citizens: Three years ago, we launched the great American comeback. Tonight, I stand before you to share the incredible results. Jobs are booming, incomes are soaring, poverty is plummeting, crime is falling, confidence is surging, and our country is thriving and highly respected again. America's enemies are on the run, America's fortunes are on the rise, and America's future is blazing bright. The years of economic decay are over. The days of our country being used, taken advantage of, and even scorned by other nations are long behind us. Gone too are the broken promises, jobless recoveries, tired platitudes, and constant excuses for the depletion of American wealth, power, and prestige. In just 3 short years, we have shattered the mentality of American decline, and we have rejected the downsizing of 

interviews


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json

# Base URL for constructing full links
base_url = 'https://www.presidency.ucsb.edu'

# URL of the main page with the list of interviews
main_url = 'https://www.presidency.ucsb.edu/documents/app-categories/presidential/interviews?items_per_page=85'

# Directory to save the JSON files
save_dir = '/content/drive/My Drive/RAG Debate Project Lab semester 8/'
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Step 1: Extract links and titles from the main page
response = requests.get(main_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all interview links
interview_links = soup.select('a[href^="/documents/interview-"]')

# Dictionary to hold interviews grouped by president
president_data = {}

for link in interview_links:
    href = link['href']
    title = link.get_text(strip=True)
    full_url = base_url + href
    page_response = requests.get(full_url)
    page_soup = BeautifulSoup(page_response.text, 'html.parser')

    # Extract the president's name
    president_name_tag = page_soup.select_one('a[href^="/people/president/"]')
    president_name = president_name_tag.get_text(strip=True) if president_name_tag else 'Unknown'

    # Extract the content (speaker and content)
    content_blocks = []
    paragraphs = page_soup.select('div.field-docs-content p')
    for para in paragraphs:
        if para.text.strip():  # Ensure we are not adding empty content
            # Extract speaker and content
            parts = para.text.split(':', 1)
            if len(parts) == 2:
                speaker, content = parts[0].strip(), parts[1].strip()
                content_blocks.append({"speaker": speaker, "content": content})

    # Construct the interview data
    interview_data = {
        "title": title,
        "president": president_name,
        "content": content_blocks
    }

    # Add to the president's data
    if president_name not in president_data:
        president_data[president_name] = []
    president_data[president_name].append(interview_data)

# Step 3: Save the data to JSON files per president
for president, interviews in president_data.items():
    file_name = f'{president.replace(" ", "_").lower()}_interviews.json'
    json_file_path = os.path.join(save_dir, file_name)
    with open(json_file_path, 'w', encoding='utf-8') as file:
        json.dump(interviews, file, ensure_ascii=False, indent=4)

    print(f"Data for {president} saved to '{json_file_path}'")

print("Data extraction completed.")


Data for Joseph R. Biden saved to '/content/drive/My Drive/RAG Debate Project Lab semester 8/joseph_r._biden_interviews.json'
Data for Donald J. Trump saved to '/content/drive/My Drive/RAG Debate Project Lab semester 8/donald_j._trump_interviews.json'
Data extraction completed.


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os

# URL of the specific page to scrape
page_url = 'https://www.presidency.ucsb.edu/documents/address-accepting-the-democratic-presidential-nomination-wilmington-delaware'

# Directory to save the JSON file
save_dir = '/content/drive/My Drive/RAG Debate Project Lab semester 8/'
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Fetch the page content
response = requests.get(page_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the title (h1 tag)
title = soup.find('h1').get_text(strip=True)

# Extract the year from the 'field-docs-start-date-time' span tag
date_span = soup.select_one('div.field-docs-start-date-time span')
year = date_span.get_text(strip=True).split()[-1] if date_span else 'Unknown'

# Extract the content (all paragraphs inside 'field-docs-content')
content_paragraphs = soup.select('div.field-docs-content p')
content = [para.get_text(strip=True) for para in content_paragraphs if para.get_text(strip=True)]

# Construct the data to be saved
data = {
    "title": title,
    "content": content,
    "year": year
}

# Save the data to a JSON file named after the h1 title
file_name = f'{title.replace(" ", "_").replace("/", "-").lower()}.json'
json_file_path = os.path.join(save_dir, file_name)
with open(json_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"Data saved to '{json_file_path}'")


Data saved to '/content/drive/My Drive/RAG Debate Project Lab semester 8/address_accepting_the_democratic_presidential_nomination_in_wilmington,_delaware.json'


statements

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json

# Base URL for constructing full links
base_url = 'https://www.presidency.ucsb.edu'

# URL of the main page with the list of statements
main_url = 'https://www.presidency.ucsb.edu/documents/app-categories/statements?items_per_page=1584'

# Directory to save the JSON files
save_dir = '/content/drive/My Drive/RAG Debate Project Lab semester 8/'
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Step 1: Extract links and titles from the main page
response = requests.get(main_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all statement links
statement_links = soup.select('a[href^="/documents/statement"]')

# Dictionary to hold statements grouped by president
president_data = {}

for link in statement_links:
    href = link['href']
    title = link.get_text(strip=True)
    full_url = base_url + href
    page_response = requests.get(full_url)
    page_soup = BeautifulSoup(page_response.text, 'html.parser')

    # Extract the president's name
    president_name_tag = page_soup.select_one('a[href^="/people/president/"]')
    president_name = president_name_tag.get_text(strip=True) if president_name_tag else 'Unknown'

    # Extract the content (all paragraphs inside 'field-docs-content')
    content_paragraphs = page_soup.select('div.field-docs-content p')
    content = [para.get_text(strip=True) for para in content_paragraphs if para.get_text(strip=True)]

    # Construct the statement data
    statement_data = {
        "title": title,
        "president": president_name,
        "content": content
    }

    # Add to the president's data
    if president_name not in president_data:
        president_data[president_name] = []
    president_data[president_name].append(statement_data)

# Step 3: Save the data to JSON files per president
for president, statements in president_data.items():
    file_name = f'{president.replace(" ", "_").replace(".", "").lower()}_statements.json'
    json_file_path = os.path.join(save_dir, file_name)
    with open(json_file_path, 'w', encoding='utf-8') as file:
        json.dump(statements, file, ensure_ascii=False, indent=4)

    print(f"Data for {president} saved to '{json_file_path}'")

print("Data extraction completed.")


Data for Joseph R. Biden saved to '/content/drive/My Drive/RAG Debate Project Lab semester 8/joseph_r_biden_statements.json'
Data for Donald J. Trump saved to '/content/drive/My Drive/RAG Debate Project Lab semester 8/donald_j_trump_statements.json'
Data extraction completed.
